In [1]:
import sys
import os

# Go two levels up to reach the root folder "ResearchAgent"
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)


In [2]:
import os
print("Current working directory:", os.getcwd())


Current working directory: d:\Projects\Research Agent\Graph


In [3]:
from langgraph.graph import StateGraph, START, END
from Graph.Nodes.state import ResearchState
from Graph.Nodes.compile_results_node import compile_results_node
from Graph.Nodes.hypothesis_generation_node import hypothesis_generation_node
from Graph.Nodes.literature_search_node import literature_search_node
from Graph.Nodes.summarization_node import summarization_node

ERROR    API status error from OpenAI API: Error code: 400 - {'error': {'message': "Invalid schema for function    
         'read_arxiv_papers': In context=('properties', 'pages_to_read'), schema must have a 'type' key.", 'type': 
         'invalid_request_error', 'param': None, 'code': None}}

WARNING  Attempt 1/1 failed: Invalid schema for function 'read_arxiv_papers': In context=('properties',            
         'pages_to_read'), schema must have a 'type' key.

ERROR    Failed after 1 attempts. Last error using OpenAIChat(o4-mini)

ERROR    Reasoning error: Invalid schema for function 'read_arxiv_papers': In context=('properties',               
         'pages_to_read'), schema must have a 'type' key.

i dont know


ERROR    API status error from OpenAI API: Error code: 400 - {'error': {'message': "Invalid schema for function    
         'read_arxiv_papers': In context=('properties', 'pages_to_read'), schema must have a 'type' key.", 'type': 
         'invalid_request_error', 'param': None, 'code': None}}

WARNING  Attempt 1/1 failed: Invalid schema for function 'read_arxiv_papers': In context=('properties',            
         'pages_to_read'), schema must have a 'type' key.

ERROR    Failed after 1 attempts. Last error using OpenAIChat(o4-mini)

ERROR    Reasoning error: Invalid schema for function 'read_arxiv_papers': In context=('properties',               
         'pages_to_read'), schema must have a 'type' key.

i dont know


In [4]:
from utils.loadapi import loadapi
loadapi()

In [5]:
def should_continue(state: ResearchState) -> str:
    """Decide if we should continue or end due to errors."""
    if state.errors:
        return "handle_errors"
    return "continue"

def handle_errors_node(state: ResearchState) -> ResearchState:
    """Create an error report."""
    error_report = "## Errors Encountered\n\n"
    for error in state.errors:
        error_report += f"- {error}\n"
    
    state.final_output = error_report
    return state

In [6]:
def create_research_workflow():
    """Create and return a LangGraph workflow for the research Process."""
    graph = StateGraph(ResearchState)
    
    # Add nodes
    graph.add_node("literature_search", literature_search_node)
    graph.add_node("summarization", summarization_node)
    graph.add_node("hypothesis_generation", hypothesis_generation_node)
    graph.add_node("compile_results", compile_results_node)
    graph.add_node("handle_errors", handle_errors_node)
    
    # Add edges
    graph.add_edge(START, "literature_search")
    graph.add_edge("literature_search", "summarization")
    graph.add_edge("summarization", "hypothesis_generation")
    graph.add_edge("hypothesis_generation", "compile_results")
    graph.add_edge("compile_results", END)
    graph.add_edge("handle_errors", END)
    
    # Add conditionanl Edges
    graph.add_conditional_edges(
        "literature_search",
        should_continue,
        {
            "continue": "summarization",
            "handle_errors": "handle_errors"
        }
    )
    
    graph.add_conditional_edges(
        "summarization",
        should_continue,
        {
            "continue": "hypothesis_generation",
            "handle_errors": "handle_errors"
        }
    )
    
    graph.add_conditional_edges(
        "hypothesis_generation",
        should_continue,
        {
            "continue": "compile_results",
            "handle_errors": "handle_errors"
        }
    )
    
    return graph

graph = create_research_workflow()
compiled_graph = graph.compile()

In [7]:
from IPython.display import Image, display

In [10]:
print(compiled_graph.get_graph().draw_ascii())

                                    +-----------+                         
                                    | __start__ |                         
                                    +-----------+                         
                                          *                               
                                          *                               
                                          *                               
                                +-------------------+                     
                                | literature_search |                     
                                +-------------------+...                  
                                   **                   .....             
                                 **                          ....         
                               **                                .....    
                    +---------------+                                 ... 
                    | sum

In [11]:
print(compiled_graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	literature_search(literature_search)
	summarization(summarization)
	hypothesis_generation(hypothesis_generation)
	compile_results(compile_results)
	handle_errors(handle_errors)
	__end__([<p>__end__</p>]):::last
	__start__ --> literature_search;
	compile_results --> __end__;
	handle_errors --> __end__;
	hypothesis_generation --> compile_results;
	literature_search --> summarization;
	summarization --> hypothesis_generation;
	literature_search -. &nbsp;continue&nbsp; .-> summarization;
	literature_search -.-> handle_errors;
	summarization -. &nbsp;continue&nbsp; .-> hypothesis_generation;
	summarization -.-> handle_errors;
	hypothesis_generation -. &nbsp;continue&nbsp; .-> compile_results;
	hypothesis_generation -.-> handle_errors;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
# Method 1: Use draw_graphviz() which doesn't require internet
display(Image(compiled_graph.get_graph().draw_mermaid_png()))

NameError: name 'Image' is not defined

In [ ]:
# Method 2: Use draw_mermaid() to get just the text syntax
mermaid_syntax = compiled_graph.get_graph().draw_mermaid()
print(mermaid_syntax)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	literature_search(literature_search)
	summarization(summarization)
	hypothesis_generation(hypothesis_generation)
	compile_results(compile_results)
	handle_errors(handle_errors)
	__end__([<p>__end__</p>]):::last
	__start__ --> literature_search;
	compile_results --> __end__;
	handle_errors --> __end__;
	hypothesis_generation --> compile_results;
	literature_search --> summarization;
	summarization --> hypothesis_generation;
	literature_search -. &nbsp;continue&nbsp; .-> summarization;
	literature_search -.-> handle_errors;
	summarization -. &nbsp;continue&nbsp; .-> hypothesis_generation;
	summarization -.-> handle_errors;
	hypothesis_generation -. &nbsp;continue&nbsp; .-> compile_results;
	hypothesis_generation -.-> handle_errors;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

# Try the pyppeteer method which uses a local browser
display(Image(compiled_graph.get_graph().draw_mermaid_png(
    draw_method=MermaidDrawMethod.PYPPETEER
)))

In [12]:
display(Image(compiled_graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)